In [1]:
%LOAD_EXT AUTORELOAD
%AUTORELOAD 2

In [2]:
'''
Description: Calculates Historical Market Wide Characteristicts
'''
from pyspark.sql import Window
import findspark
from pyspark import StorageLevel

from ams.services import spark_service
from ams.config import constants
from ams.services.ticker_service import  get_nasdaq_tickers

from ams.config import logger_factory

logger = logger_factory.create("market_roi.ipynb")

In [3]:
import os
os.environ['OBJC_DISABLE_INITIALIZE_FORK_SAFETY'] = 'YES'

findspark.init()
spark = spark_service.get_or_create(app_name='roi')
sc = spark.sparkContext
logger.info("pyspark script logger initialized")

19:04:32 - ams.services.spark_service:29 - INFO - http://DESKTOP-RI98UNH.mshome.net:4040
19:04:33 - ams.services.spark_service:29 - INFO - http://DESKTOP-RI98UNH.mshome.net:4040
19:04:33 - market_roi.ipynb:7 - INFO - pyspark script logger initialized


In [71]:
from pyspark.sql import types as T

schema_ticker = T.StructType(fields=[T.StructField('ticker', T.StringType()),
                            T.StructField('date', T.StringType()),
                            T.StructField('open', T.FloatType()),
                            T.StructField('high', T.FloatType()),
                            T.StructField('low', T.FloatType()),
                            T.StructField('close', T.FloatType()),
                            T.StructField('volume', T.FloatType()),
                            T.StructField('dividends', T.FloatType()),
                            T.StructField('closeunadj', T.FloatType()),
                            T.StructField('lastupdated', T.StringType()),
                            T.StructField('closeadj', T.FloatType()),
                            ])

df = spark.read.schema(schema_ticker).csv(str(constants.SHAR_SPLIT_EQUITY_EOD_DIR), header=True)
df_dropped = df.drop("None")
df_dropped = df_dropped.select("ticker", "date", "close")

In [72]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

df_d = df_dropped

ticker_window = Window.partitionBy("ticker").orderBy("date")
df_d = df_d.withColumn("close_1_day_before", F.lag("close", 1).over(ticker_window))
df_d = df_d.withColumn("close_2_day_before", F.lag("close", 2).over(ticker_window))
df_d = df_d.withColumn("close_3_day_before", F.lag("close", 3).over(ticker_window))
df_d = df_d.withColumn("close_4_day_before", F.lag("close", 4).over(ticker_window))
df_d = df_d.withColumn("close_5_day_before", F.lag("close", 5).over(ticker_window))

df_d.count()

20011018

In [73]:
df_apple = df_d.filter(F.col("ticker") == "AAPL")
df_apple.createOrReplaceTempView("apple")

df_apple_lim = spark.sql("""
SELECT * from apple limit 10""")

df_apple_lim.toPandas().head()

,ticker,date,close,close_1_day_before,close_2_day_before,close_3_day_before,close_4_day_before,close_5_day_before
0,AAPL,2009-01-02,12.964,NaN,NaN,NaN,NaN,NaN
1,AAPL,2009-01-05,13.511,12.964,NaN,NaN,NaN,NaN
2,AAPL,2009-01-06,13.289,13.511,12.964,NaN,NaN,NaN
3,AAPL,2009-01-07,13.001,13.289,13.511,12.964,NaN,NaN
4,AAPL,2009-01-08,13.243,13.001,13.289,13.511,12.964,NaN


In [74]:
import twitter_udf

df_roi = df_d

df_roi = df_roi.withColumn("1_day_roi", twitter_udf.calc_roi(F.col("close_1_day_before"), F.col("close")))
df_roi = df_roi.withColumn("2_day_roi", twitter_udf.calc_roi(F.col("close_2_day_before"), F.col("close")))
df_roi = df_roi.withColumn("3_day_roi", twitter_udf.calc_roi(F.col("close_3_day_before"), F.col("close")))
df_roi = df_roi.withColumn("4_day_roi", twitter_udf.calc_roi(F.col("close_4_day_before"), F.col("close")))
df_roi = df_roi.withColumn("5_day_roi", twitter_udf.calc_roi(F.col("close_5_day_before"), F.col("close")))

df_roi.createOrReplaceTempView("eod_ticker")

In [75]:
df_limit = spark.sql("""SELECT * from eod_ticker order by ticker limit 10""")

In [76]:
df_limit.toPandas().head()

,ticker,date,close,close_1_day_before,close_2_day_before,close_3_day_before,close_4_day_before,close_5_day_before,1_day_roi,2_day_roi,3_day_roi,4_day_roi,5_day_roi
0,A,2009-01-14,18.450001,19.059999,18.520000,18.520000,18.35,18.230000,-0.032004,-0.003780,-0.003780,0.005450,0.012068
1,A,2009-01-08,18.350000,18.230000,18.129999,16.790001,16.24,NaN,0.006583,0.012135,0.092912,0.129926,NaN
2,A,2009-01-13,19.059999,18.520000,18.520000,18.350000,18.23,18.129999,0.029158,0.029158,0.038692,0.045529,0.051296
3,A,2009-01-06,18.129999,16.790001,16.240000,NaN,NaN,NaN,0.079809,0.116379,NaN,NaN,NaN
4,A,2009-01-07,18.230000,18.129999,16.790001,16.240000,NaN,NaN,0.005516,0.085765,0.122537,NaN,NaN


In [77]:
df_sma = df_roi
sma_num = [5, 10, 15, 20, 50, 100, 200]

df_sma = df_sma.withColumn('days_from_epoch', twitter_udf.get_days_from_epoch(F.col("date")))

for i in sma_num:
    rolling_ticker = Window.partitionBy("ticker").orderBy("days_from_epoch").rangeBetween(-(i + 1), -1)
    col_new = f"SMA_{i}"
    df_sma = df_sma.withColumn(col_new, F.avg(F.col("close")).over(rolling_ticker))
    cols_to_save.append(col_new)

df_std.persist()
df_sma.count()

20011018

In [78]:
df_std = df_sma

std_rolling = 100
rolling_ticker = Window.partitionBy("ticker").orderBy("days_from_epoch").rangeBetween(-(std_rolling + 1), -1)
col_new = f"STD_{std_rolling}"
df_std = df_std.withColumn(col_new, F.stddev(F.col("close")).over(ticker_window))

df_std.persist()
df_std.count()

20011018

In [79]:
df_std.write.format("parquet").mode("overwrite").save(str(constants.STOCK_AGG_DATAFILE))